# Instructions


1.   Make sure that your XLSForm has a column with english wording that is called ***label::English***;all of these sheets: `survey`,`choices`,`settings`.
2. Rename your XLSForm to `thefile.xlsx` and upload it in the directory.
2.   Upload your Google translate API authentication JSON file in the directory and rename it to `authentication.json`. You can get a free API key here:
https://cloud.google.com/docs/authentication/api-keys#console
4. Enter your desired output language code (eg. `fr` for French) in the next cell. A list of codes can be found here: https://developers.google.com/admin-sdk/directory/v1/languages
```
Note: You can only translate into one language at once. If you want to translate multiple languages you can either repeat the process or modify the code to add a loop in Cell 5.
```
5. Execute all cells.






In [ ]:
# Set your desired target language:
target_language = 'de'

In [ ]:
# Install dependencies
!pip install openpyxl google-cloud-translate pandas

In [ ]:
# Import modules
from google.cloud import translate_v2 as translate
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import os

In [ ]:
# Upload your XLSForm document and insert the path below:
uploaded_file = "thefile.xlsx"
#Extract the sheets to pandas datafreames
dfchoices = pd.read_excel(uploaded_file, 'choices', engine = 'openpyxl')
dfsurvey = pd.read_excel(uploaded_file, 'survey', engine = 'openpyxl')
dfsettings = pd.read_excel(uploaded_file, 'settings', engine = 'openpyxl')

In [ ]:
# The translation
import os
import pandas as pd
from google.cloud import translate_v2 as translate

# Set your API key
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'thefile.json'

translate_client = translate.Client()
label = "label::English"


def translate_text(text, target_language):
    if text == "":
        return ""
    result = translate_client.translate(text, target_language)
    return result['translatedText']

def translate_dataframe(df, column, chunk_size=50):
    translated_labels = []
    for i in range(0, len(df), chunk_size):
        text_chunk = df[column][i:i + chunk_size].apply(str).tolist()  # Convert non-string items to strings
        text_to_translate = '<sep>'.join(text_chunk)

        translated_text = translate_text(text_to_translate, target_language)
        translated_labels_chunk = translated_text.split('<sep>')

        translated_labels.extend(translated_labels_chunk)

    # Assign translated labels to the new column
    if len(translated_labels) == len(df):
        df[f'label::{target_language}'] = translated_labels
    else:
        print(f"Error: Length of translated labels does not match the length of the original text in {column} column.")

    # Convert "nan" strings to NaN (missing values)
    df[f'label::{target_language}'] = df[f'label::{target_language}'].replace('Nan', pd.NA)

    return df

# Assuming you have a DataFrame called 'dfsurvey' with the "label::English" column
dfsurvey = translate_dataframe(dfsurvey, label)
# Assuming you have a DataFrame called 'dfchoices' with the "label::English" column
dfchoices = translate_dataframe(dfchoices, label)


In [ ]:
#Saving new XLSForm
import pandas as pd

# Create a Pandas Excel writer using openpyxl as the engine
name = 'Translated Survey'
file_name = f'{name}.xlsx'

with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
    # Write each dataframe to a different worksheet
    dfsurvey.to_excel(writer, sheet_name='survey', index=False)
    dfchoices.to_excel(writer, sheet_name='choices', index=False)
    dfsettings.to_excel(writer, sheet_name='settings', index=False)

